<a href="https://colab.research.google.com/github/Joongeun/Internship/blob/Topic-Modeling/Copy_of_XLM_T_Run_a_classifier_on_a_text_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installs and imports

In [1]:
!pip install --upgrade pip
!pip install sentencepiece
!pip install transformers
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import AutoModelForSequenceClassification
from torch.utils.data import DataLoader
import numpy as np
from scipy.special import softmax
import spacy

In [4]:
import json

nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words

posts = []

with open('/content/QuitVaping_submissions', 'r', encoding='utf-8') as f:
  while True:
    try:
      data = json.loads(f.readline())
      text = [word for word in data['selftext'].split() if not word in stop_words]
      posts.append(' '.join(text))
      #posts.append(data['selftext'])
    except:
      break

posts = list(set(posts))
# Some deleted/remoed posts have other content in it --> check if "[removed]" or "[deleted]" is in the post (e.x: deleted polls have links along with the word "[deleted]")
for i in posts:
  if "[removed]" in i or "[deleted]" in i or len(i) < 1:
    posts.remove(i)
print(len(posts))

7796


## Data

In [5]:
def preprocess(corpus):
  outcorpus = []
  for text in corpus:
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    new_text = " ".join(new_text)
    outcorpus.append(new_text)
  return outcorpus

In [ ]:
!wget https://raw.githubusercontent.com/cardiffnlp/xlm-t/main/data/sentiment/all/test_text.txt

In [6]:
dataset_path = './test_text.txt'
dataset = posts

In [7]:
# this is a dataset in 8 different languages
for example in [0,870,1740,2610,3480,4350,5220,6090]:
  print(dataset[example])

Y’all, I want vape bad. If I’m going honest I don’t want quit, I know I need to. Well, that’s lie I hate I’m addicted, I want feel like that… anxiety, fiending, restlessness…but I don’t want negative effects nic… I want nic. I feel body bad it’s craving it, it’s insane. Nothing satisfy it.. weed, food, gum, gym, spending time outside, spending time friends don’t vape. I woke SUPER proud day 2, I thought I’d cave night I dream I broke vape n I super upset—while disappointed I hit vape. So I’m struggling internally lmao. Here theory: I’m hungry bc I know DOES things me—when I’m stressed example. So times I wanna vape I’m bored, driving, I don’t hit vape—discipline it’s craving use gum something, it’s stress related find good replacement don’t mad I choose vape? Could I vape pacifier sense of, hey don’t need pacifier day anymore ur rly rly rly upset I’ll like parents kids? Lmao. I dont know way honestly. I tapering, n working okay I’d think abt disposable, I’m going cold turkey it’s preoc

## Model

In [8]:
CUDA = True # set to true if using GPU (Runtime -> Change runtime Type -> GPU)
BATCH_SIZE = 4
MODEL = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True)
config = AutoConfig.from_pretrained(MODEL) # used for id to label name
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
if CUDA:
  model = model.to('cuda')
_ = model.eval()

## Forward

In [9]:
def forward(text, cuda=True):
  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
  if cuda:
    encoded_input.to('cuda')
    output = model(**encoded_input)
    scores = output[0].detach().cpu().numpy()
  else:
    output = model(**encoded_input)
    scores = output[0].detach().numpy()

  scores = softmax(scores, axis=-1)
  return scores

In [10]:
dl = DataLoader(dataset, batch_size=BATCH_SIZE)
all_preds = []
for idx,batch in enumerate(dl):
  print('Batch ',idx+1,' of ',len(dl))
  text = preprocess(batch)
  try:
    scores = forward(text, cuda=CUDA)
  except:
    continue
  preds = np.argmax(scores, axis=-1)
  all_preds.extend(preds)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Batch  1  of  1949
Batch  2  of  1949
Batch  3  of  1949
Batch  4  of  1949
Batch  5  of  1949
Batch  6  of  1949
Batch  7  of  1949
Batch  8  of  1949
Batch  9  of  1949
Batch  10  of  1949
Batch  11  of  1949
Batch  12  of  1949
Batch  13  of  1949
Batch  14  of  1949
Batch  15  of  1949
Batch  16  of  1949
Batch  17  of  1949
Batch  18  of  1949
Batch  19  of  1949
Batch  20  of  1949
Batch  21  of  1949
Batch  22  of  1949
Batch  23  of  1949
Batch  24  of  1949
Batch  25  of  1949
Batch  26  of  1949
Batch  27  of  1949
Batch  28  of  1949
Batch  29  of  1949
Batch  30  of  1949
Batch  31  of  1949
Batch  32  of  1949
Batch  33  of  1949
Batch  34  of  1949
Batch  35  of  1949
Batch  36  of  1949
Batch  37  of  1949
Batch  38  of  1949
Batch  39  of  1949
Batch  40  of  1949
Batch  41  of  1949
Batch  42  of  1949
Batch  43  of  1949
Batch  44  of  1949
Batch  45  of  1949
Batch  46  of  1949
Batch  47  of  1949
Batch  48  of  1949
Batch  49  of  1949
Batch  50  of  1949
Batch  51

In [11]:
print(len(dataset))

7796


In [12]:
# this is a dataset in 8 different languages
#all_preds is list of 0s (neut), 1s (neg), 2s (pos)
print(len(all_preds))
for example in [0,870,1740,2610]:
  pred = all_preds[example]
  print(pred)
  print(dataset[example], '--->', config.id2label[pred])

3348
0
Y’all, I want vape bad. If I’m going honest I don’t want quit, I know I need to. Well, that’s lie I hate I’m addicted, I want feel like that… anxiety, fiending, restlessness…but I don’t want negative effects nic… I want nic. I feel body bad it’s craving it, it’s insane. Nothing satisfy it.. weed, food, gum, gym, spending time outside, spending time friends don’t vape. I woke SUPER proud day 2, I thought I’d cave night I dream I broke vape n I super upset—while disappointed I hit vape. So I’m struggling internally lmao. Here theory: I’m hungry bc I know DOES things me—when I’m stressed example. So times I wanna vape I’m bored, driving, I don’t hit vape—discipline it’s craving use gum something, it’s stress related find good replacement don’t mad I choose vape? Could I vape pacifier sense of, hey don’t need pacifier day anymore ur rly rly rly upset I’ll like parents kids? Lmao. I dont know way honestly. I tapering, n working okay I’d think abt disposable, I’m going cold turkey it’

In [13]:
counter = {0: 0, 1: 0, 2: 0}
labels = list(counter.keys())
for i in all_preds:
  counter[labels[i]] += 1
print(counter)
avg = lambda x: x/len(all_preds)
for i in labels:
  print(i, avg(counter[i]))

{0: 2630, 1: 398, 2: 320}
0 0.7855436081242533
1 0.11887694145758662
2 0.0955794504181601
